<a href="https://colab.research.google.com/github/viniciusrpb/cloud_image_segmentation/blob/main/cloud_classification_ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloud Classification using Vision Transformers

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!cp -r "/content/drive/My Drive/img_satelite/CCSN/train" "training"
#!cp -r "/content/drive/My Drive/img_satelite/CCSN/val" "validation"
#!cp -r "/content/drive/My Drive/img_satelite/CCSN/test" "testing"

In [ ]:
#!pip install -U tensorflow-addons
#!pip install -U torch torch-vision
#!pip install datasets transformers

In [ ]:
from transformers import ViTFeatureExtractor
from transformers import ViTForImageClassification
from PIL import Image, UnidentifiedImageError
import matplotlib.pyplot as plt
import numpy as np
from datasets import load_metric
from datasets import load_dataset
from pathlib import Path
from torchvision.datasets import ImageFolder

In [ ]:
path_train = Path('training')
path_validation = Path('validation')
path_test = Path('testing')

In [ ]:
training_set_ds = ImageFolder(path_train)
validation_set_ds = ImageFolder(path_validation)
testing_set_ds = ImageFolder(path_test)

In [ ]:
plt.figure(figsize=(100,50))
num_examples_per_class = 1
i = 1
for class_idx, class_name in enumerate(training_set_ds.classes):
  folder = training_set_ds.root / class_name
  for image_idx, image_path in enumerate(sorted(folder.glob('*'))):
      print(image_path)
      if image_path.suffix in training_set_ds.root:
        image = Image.open(image_path)
        plt.subplot(len(training_set_ds.classes), num_examples_per_class, i)
        ax = plt.gca()
        ax.set_title(
            class_name,
            size='xx-large',
            pad=5,
            loc='left',
            y=0,
            backgroundcolor='white'
        )
        ax.axis('off')
        plt.imshow(image)
        i += 1
        if image_idx + 1 == num_examples_per_class:
            break


training/Ac/Ac-N001.jpg


TypeError: ignored

<Figure size 7200x3600 with 0 Axes>

Allocate objects for loading the data using the DataGenerator

In [ ]:
metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)


labels = ds['train'].features['labels'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

NameError: ignored

hyperparameters

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-beans",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor,
)

Create the MLP

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Criação dos patches

In [ ]:
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

Vision Transformer Encoder-Decoder Architecture

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(4, 4))

x_batch, y_batch = next(train_generator)
image = x_batch[0]
plt.imshow(image)
plt.axis("off")

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]), size=(image_size, image_size)
)
patches = Patches(patch_size)(resized_image)
print(f"Image size: {image_size} X {image_size}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 3))
    plt.imshow(patch_img)
    plt.axis("off")

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

Define o Modelo

In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    #augmented = data_augmentation(inputs)
    augmented = inputs
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [ ]:
model = create_vit_classifier()

huggingface_model = 'google/vit-base-patch16-224-in21k'

optimizer = tfa.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay)

f1_score = tfa.metrics.F1Score(num_classes=8, average='macro',threshold=0.5)

model.compile(optimizer=optimizer,loss = 'categorical_crossentropy',
              metrics = ['accuracy',f1_score])

checkpoint_filepath = "/tmp/checkpoint"
checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

In [ ]:
history_fine = model.fit(train_generator,
                         epochs=num_epochs,
                         batch_size=batch_size,
                         validation_data=validation_generator,
                         callbacks=[checkpoint_callback]
                         )

Prediction using the test set

In [ ]:
for image, target in ds_test.take(1):
  y = model.predict(image)
  print(np.argmax(y, axis=1), target)

In [ ]:
score = model.evaluate(test_generator,batch_size=batch_size) 
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

To improve the model results, try to train with
more epochs
different image size
more number of TransformerBlocks
change the patch size
change the learning rate, optimizer, weight decay
